# Agents

AI agents are software systems that use AI to perform tasks on behalf of users. In this notebook, we'll experiment with creating and running AI agents using two different frameworks: OpenAI Agents and Pydantic AI.

## Prerequisites

Follow the README instructions to set up your Python environment.

For the code examples that use local models, we'll be using Ollama's `llama3.2` model. If you haven't done so already, [install Ollama](https://ollama.com/download) on your computer and download the model with `ollama pull llama3.2`.

## OpenAI Agents SDK

The [OpenAI Agents SDK](https://github.com/openai/openai-agents-python/tree/main?tab=readme-ov-file#functions-example) is a tool that allows you to build agent-based workflows.

This example shows how to build a simple agent that can call a function and return the result. Less code is required, and it automatically handles the conversation history using `last_response_id`.

In [1]:
import random

from agents import Agent, Runner, function_tool


@function_tool
def get_weather(city: str) -> dict:
    """Get the weather in a given city."""
    random_weather = random.choice(
        [
            "sunny",
            "cloudy",
            "rainy",
            "snowy",
            "partly cloudy",
            "stormy",
        ]
    )
    random_temp = random.randint(10, 100)

    return {
        "conditions": random_weather,
        "temperature": random_temp,
        "city": city,
        "units": {"temperature": "fahrenheit"},
    }


# Create the agent with the weather tool
agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful weather agent. When asked about weather, use the get_weather tool to provide accurate information.",
    tools=[get_weather],
)


async def main():
    """Main function to run the agent with conversation history."""
    # First query
    result1 = await Runner.run(
        agent, input="What's the weather in Tokyo?"
    )
    print("Query 1:", "What's the weather in Tokyo?")
    print("Response:", result1.final_output)

    print("\n" + "=" * 50 + "\n")

    # Second query - pass the previous response ID to maintain conversation
    result2 = await Runner.run(
        agent,
        input="How about the weather in New York?",
        previous_response_id=result1.last_response_id,
    )
    print("Query 2:", "How about the weather in New York?")
    print("Response:", result2.final_output)

    print("\n" + "=" * 50 + "\n")

    # Third query - pass the previous response ID to maintain conversation
    result3 = await Runner.run(
        agent,
        input="What cities did I ask about?",
        previous_response_id=result2.last_response_id,
    )
    print("Query 3:", "What cities did I ask about?")
    print("Response:", result3.final_output)


await main()

Query 1: What's the weather in Tokyo?
Response: The weather in Tokyo is currently stormy with a temperature of 46°F. If you need more details or a forecast, just let me know!


Query 2: How about the weather in New York?
Response: The weather in New York is currently snowy with a temperature of 78°F. Let me know if you need more details or a forecast!


Query 3: What cities did I ask about?
Response: You asked about the weather in the following cities:

1. Tokyo
2. New York

If you want information about more cities, feel free to ask!


### Maintaining conversation history using a `Session`

You can also use a `Session` object to maintain conversation history across multiple agent runs, instead of passing `previous_response_id` to `Runner.run()`. In the example below, we'll store the conversation history in an in-memory SQLite database, but you can pass a `db_path` argument to `SQLiteSession` for persistent storage. You can also implement your own session memory by creating a class that follows the `Session` protocol.

In [2]:
from agents import SQLiteSession

# Create a session instance
session = SQLiteSession(session_id="my_session_id")


async def main():
    """Main function to run the agent with conversation history."""
    # First query
    result1 = await Runner.run(
        agent,
        input="What's the weather in Tokyo?",
        session=session,
    )
    print("Query 1:", "What's the weather in Tokyo?")
    print("Response:", result1.final_output)

    print("\n" + "=" * 50 + "\n")

    # Second query
    result2 = await Runner.run(
        agent,
        input="How about the weather in New York?",
        session=session,
    )
    print("Query 2:", "How about the weather in New York?")
    print("Response:", result2.final_output)

    print("\n" + "=" * 50 + "\n")

    # Third query
    result3 = await Runner.run(
        agent,
        input="What cities did I ask about?",
        session=session,
    )
    print("Query 3:", "What cities did I ask about?")
    print("Response:", result3.final_output)


await main()

Query 1: What's the weather in Tokyo?
Response: The current weather in Tokyo is cloudy with a temperature of 73°F. If you need more details such as humidity or forecast, let me know!


Query 2: How about the weather in New York?
Response: The weather in New York is currently cloudy with a temperature of 81°F. If you’d like more weather details or a forecast, just let me know!


Query 3: What cities did I ask about?
Response: You asked about the weather in two cities: Tokyo and New York. If you want information about any other cities, just let me know!


### Using local models

The SDK's integration with [LiteLLM](https://docs.litellm.ai/) allows us to use models from many other providers, including local Ollama models. We will use the same code as before, but provide an Ollama model to the `Agent` constructor.

In [3]:
from agents import set_tracing_disabled
from agents.extensions.models.litellm_provider import (
    LitellmProvider,
)

# Disable tracing. By default, the SDK will send information about agent runs to
# OpenAI. We don't necessarily want this when using local models.
set_tracing_disabled(True)

# Create a session instance
session = SQLiteSession(session_id="my_session_id")

# Create the agent with the weather tool
agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful weather agent. When asked about weather, use the get_weather tool to provide accurate information.",
    tools=[get_weather],
    model=LitellmProvider().get_model(
        "ollama_chat/llama3.2"
    ),
)


async def main():
    """Main function to run the agent with conversation history."""
    # First query
    result1 = await Runner.run(
        agent,
        input="What's the weather in Tokyo?",
        session=session,
    )
    print("Query 1:", "What's the weather in Tokyo?")
    print("Response:", result1.final_output)

    print("\n" + "=" * 50 + "\n")

    # Second query
    result2 = await Runner.run(
        agent,
        input="How about the weather in New York?",
        session=session,
    )
    print("Query 2:", "How about the weather in New York?")
    print("Response:", result2.final_output)

    print("\n" + "=" * 50 + "\n")

    # Third query
    result3 = await Runner.run(
        agent,
        input="What cities did I ask about?",
        session=session,
    )
    print("Query 3:", "What cities did I ask about?")
    print("Response:", result3.final_output)


await main()

Query 1: What's the weather in Tokyo?
Response: The current weather in Tokyo is cloudy with a temperature of 37°F.


Query 2: How about the weather in New York?
Response: The current weather in New York is cloudy with a temperature of 98°F.


Query 3: What cities did I ask about?
Response: The cities you asked about are Tokyo and New York.


## Pydantic AI Agents

[Pydantic AI](https://ai.pydantic.dev/) is another framework for creating and running AI agents. It also supports many different model providers, and its API is kinda similar to the OpenAI Agents SDK.

Let's use it to recreate the same weather agent we created before, using OpenAI's `gpt-5-nano` model.

In [4]:
from pydantic_ai import Agent


def get_weather(city: str) -> dict:
    """Get the weather in a given city."""
    random_weather = random.choice(
        [
            "sunny",
            "cloudy",
            "rainy",
            "snowy",
            "partly cloudy",
            "stormy",
        ]
    )
    random_temp = random.randint(10, 100)

    return {
        "conditions": random_weather,
        "temperature": random_temp,
        "city": city,
        "units": {"temperature": "fahrenheit"},
    }


# Create the agent with the weather tool
agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful weather agent. When asked about weather, use the get_weather tool to provide accurate information.",
    tools=[get_weather],
    model="openai:gpt-5-nano",
)


async def main():
    """Main function to run the agent with conversation history."""
    # First query
    result1 = await agent.run(
        "What's the weather in Tokyo?"
    )
    print("Query 1:", "What's the weather in Tokyo?")
    print("Response:", result1.output)

    print("\n" + "=" * 50 + "\n")

    # Second query - pass the message history so far to maintain conversation
    result2 = await agent.run(
        "How about the weather in New York?",
        message_history=result1.all_messages(),
    )
    print("Query 2:", "How about the weather in New York?")
    print("Response:", result2.output)

    print("\n" + "=" * 50 + "\n")

    # Third query - pass the message history so far to maintain conversation
    result3 = await agent.run(
        "What cities did I ask about?",
        message_history=result2.all_messages(),
    )
    print("Query 3:", "What cities did I ask about?")
    print("Response:", result3.output)


await main()

Query 1: What's the weather in Tokyo?
Response: Tokyo — snowy. Temperature: 51°F (about 11°C).

Tips: Snow can make roads and sidewalks slick—dress warmly and check transit updates if you’re traveling.

Would you like an hourly forecast or a Celsius conversion?


Query 2: How about the weather in New York?
Response: New York — stormy, 94°F (about 34°C).

Would you like an hourly forecast or a Celsius conversion?


Query 3: What cities did I ask about?
Response: Tokyo and New York.


### Using local models

We can also create our agent using an Ollama model. We'll only change the `model` argument in the `Agent` constructor.

In [5]:
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider


# Create the agent with the weather tool
# We could also set the `model` to OpenAIChatModel(model_name="llama3.2", provider="ollama")
# if we have a OLLAMA_BASE_URL environment variable (e.g. OLLAMA_BASE_URL=http://localhost:11434/v1)
agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful weather agent. When asked about weather, use the get_weather tool to provide accurate information.",
    tools=[get_weather],
    model=OpenAIChatModel(
        model_name="llama3.2",
        provider=OllamaProvider(
            base_url="http://localhost:11434/v1"
        ),
    ),
)


async def main():
    """Main function to run the agent with conversation history."""
    # First query
    result1 = await agent.run(
        "What's the weather in Tokyo?"
    )
    print("Query 1:", "What's the weather in Tokyo?")
    print("Response:", result1.output)

    print("\n" + "=" * 50 + "\n")

    # Second query - pass the message history so far to maintain conversation
    result2 = await agent.run(
        "How about the weather in New York?",
        message_history=result1.all_messages(),
    )
    print("Query 2:", "How about the weather in New York?")
    print("Response:", result2.output)

    print("\n" + "=" * 50 + "\n")

    # Third query - pass the message history so far to maintain conversation
    result3 = await agent.run(
        "What cities did I ask about?",
        message_history=result2.all_messages(),
    )
    print("Query 3:", "What cities did I ask about?")
    print("Response:", result3.output)


await main()

Query 1: What's the weather in Tokyo?
Response: The current weather in Tokyo is mostly cloudy, with a temperature of 81°F.


Query 2: How about the weather in New York?
Response: The current weather in New York is sunny, with a temperature of 57°F.


Query 3: What cities did I ask about?
Response: You asked about the weather in Tokyo and New York.
